## BERT 嵌入生成

安装Transformers库，这里使用的版本是3.5.1

In [1]:
! pip install Transformers==3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 20.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


### BERT的顶层编码器（编码器12）获得嵌入

In [2]:
from transformers import BertModel, BertTokenizer
import torch

下载预训练的BERT模型，使用的是不区分大小写的模型

In [3]:
model = BertModel.from_pretrained('bert-base-uncased')  # 下载并加载预训练模型
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # 下载并加载用于预训练模型的词元分析器

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


#### 输入预处理

In [4]:
sentence = 'I love Paris'
# 对句子进行分词
tokens = tokenizer.tokenize(sentence)
print(tokens)
# 添加[CLS]和[SEP]
tokens = ['[CLS]'] + tokens + ['[SEP]']
print(tokens)
# 统一长度为7
tokens = tokens + ['[PAD]'] + ['[PAD]']
print(tokens)
# 获取注意力掩码
attention_mask = [1 if i != '[PAD]' else 0 for i in tokens]
print(attention_mask)
# 获取标记ID
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

['i', 'love', 'paris']
['[CLS]', 'i', 'love', 'paris', '[SEP]']
['[CLS]', 'i', 'love', 'paris', '[SEP]', '[PAD]', '[PAD]']
[1, 1, 1, 1, 1, 0, 0]
[101, 1045, 2293, 3000, 102, 0, 0]


`unsqueeze()` 的目的是改变张量的形状，增加一个维度。下面两个都将变为 1 * 7 的二维矩阵。

In [5]:
# 将注意力掩码和标记ID转化为张量，方便后续计算
token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)
print(token_ids.shape)
print(attention_mask.shape)

torch.Size([1, 7])
torch.Size([1, 7])


#### 获取嵌入向量

In [6]:
outputs = model(token_ids, attention_mask=attention_mask)
# 提取 hidden_rep 和 cls_head
hidden_rep = outputs.last_hidden_state  # 形状为 [batch_size, sequence_length, hidden_size]
cls_head = outputs.pooler_output        # 形状为 [batch_size, hidden_size]
print(hidden_rep.shape)  # 这是所有 token 的最后一层隐状态
print(cls_head.shape)    # 这是 [CLS] token 的池化输出

torch.Size([1, 7, 768])
torch.Size([1, 768])


第1个值`hidden_rep`表示隐藏状态的特征，它包括从顶层编码器（编码器12）获得的所有标记的特征。第2个值`cls_head`表示`[CLS]`标记的特征
- `hidden_rep[0][0]`给出了第1个标记[CLS]的特征。
- `hidden_rep[0][1]`给出了第2个标记`I`的特征。
- `hidden_rep[0][2]`给出了第3个标记`love`的特征。

`cls_head`作为句子I love Paris的整句特征

### BERT的所有编码器层获得嵌入

In [7]:
from transformers import BertModel, BertTokenizer
import torch

下载预训练的BERT模型和词元分析器

In [8]:
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True) # 允许我们从所有编码层获得嵌入
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#### 输入预处理

In [9]:
sentence = 'I love Paris'
tokens = tokenizer.tokenize(sentence)
tokens = ['[CLS]'] + tokens + ['[SEP]']
tokens = tokens + ['[PAD]'] + ['[PAD]']

attention_mask = [1 if i != '[PAD]' else 0 for i in tokens]
token_ids = tokenizer.convert_tokens_to_ids(tokens)

token_ids = torch.tensor(token_ids).unsqueeze(0)
attention_mask = torch.tensor(attention_mask).unsqueeze(0)

#### 获取嵌入

In [10]:
outputs = model(token_ids, attention_mask=attention_mask)
last_hidden_state = outputs.last_hidden_state
pooler_output = outputs.pooler_output
hidden_states = outputs.hidden_states

- `last_hidden_state`包含从最后的编码器（编码器12）中获得的所有标记的特征。
- `pooler_output`表示来自最后的编码器的[CLS]标记的特征，它被一个线性激活函数和tanh激活函数进一步处理。
- `hidden_states`包含从所有编码器层获得的所有标记的特征

In [11]:
print(last_hidden_state.shape)

torch.Size([1, 7, 768])


- `last_hidden_state[0][0]`给出了第1个标记[CLS]的特征。
- `last_hidden_state[0][1]`给出了第2个标记`I`的特征。
- `last_hidden_state[0][2]`给出了第3个标记`love`的特征。

In [12]:
print(pooler_output.shape)

torch.Size([1, 768])


`pooler_output`作为句子I love Paris的整句特征

In [13]:
len(hidden_states)

13

- `hidden_states[0]`包含从输入嵌入层获得的所有标记的特征。
- `hidden_states[1]`包含从第1个编码器层获得的所有标记的特征。
- `hidden_states[2]`包含从第2个编码器层获得的所有标记的特征。
- `hidden_states[12]`包含从最后一个编码器层获得的所有标记的特征。

In [14]:
print(hidden_states[0].shape)
print(hidden_states[1].shape)

torch.Size([1, 7, 768])
torch.Size([1, 7, 768])
